In [25]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from itertools import product, combinations, permutations
#%%!from scikit import 


In [26]:
# Import player data and historical results for all countries
raw_data_path = 'data/'

prior = pd.read_csv(raw_data_path + 'results.csv')
fifa_dataset = pd.read_csv(raw_data_path + 'CompleteDataset.csv')
schedule = pd.read_csv(raw_data_path + 'schedule-fifa-wc-2018.csv')
matchdata = pd.read_csv(raw_data_path + 'WC18_competition_stats.csv')

# drop unwanted fields from both data sets
# fifa_dataset.drop(labels = ['Unnamed: 0', 'Age', 'Photo', 'Flag', 'Club', 'Club Logo', 'Wage'], axis = 1, inplace = True)
prior.drop(labels = ['city', 'country', 'tournament'], axis = 1, inplace = True)
prior = prior[prior.date > '1990-01-01']


# only interested in countries that are qualified for the WC
# maybe reduce this to 16 countries post group stage for faster computation (other 2^32 simulations)
qualified_countries = ['Egypt', 'Morocco', 'Nigeria', 'Senegal', 'Tunisia', 'Australia', 'Iran', 'Japan', 'Korea Republic', 'Saudi Arabia', 'Belgium', 'Croatia', 'Denmark', 'England', 'France', 'Germany', 'Iceland', 'Poland', 'Portugal', 'Russia', 'Serbia', 'Spain', 'Sweden', 'Switzerland', 'Costa Rica', 'Mexico', 'Panama', 'Argentina', 'Brazil', 'Colombia', 'Peru', 'Uruguay']

groupA = ['Russia'   , 'Saudi Arabia', 'Egypt'     , 'Uruguay']
groupB = ['Portugal' , 'Spain'       , 'Morocco'   , 'Iran']
groupC = ['France'   , 'Australia'   , 'Peru'      , 'Denmark']
groupD = ['Argentina', 'Iceland'     , 'Croatia'   , 'Nigeria']
groupE = ['Brazil'   , 'Switzerland' , 'Costa Rica', 'Serbia' ]
groupF = ['Germany'  , 'Mexico'      , 'Sweden'    , 'Korea Republic']
groupG = ['Belgium'  , 'Panama'      , 'Tunisia'   , 'England']
groupH = ['Poland'   , 'Senegal'     , 'Colombia'  , 'Japan']

groups = [groupA, groupB, groupC, groupD, groupE, groupF, groupG, groupH]

In [27]:
data = matchdata.iloc[:, range(1,11)]
targets = matchdata.iloc[:, -1]

gnb = GaussianNB()
model = gnb.fit(data, targets)

prob_matrix = np.zeros((32,32))
#For each pair of teams, find predicted label, convert to probability.

teamdata = pd.read_csv('data/WC18_processed.csv')


for i in range(0,32):
    for j in range(i,32):
        teamA = teamdata.iloc[i,:]
        teamB = teamdata.iloc[j,:]

        point = [teamA.Attack, teamB.Attack, teamA.Defence, teamB.Defence, teamA.Midfield, teamB.Midfield, teamA.Rating, teamB.Rating, teamA.Tier, teamB.Tier]
        df = pd.DataFrame(np.array(point).reshape((1,10)), columns=data.columns)
        winner = gnb.predict(df)
        probs = gnb.predict_proba(df)

        prob_matrix[i,j] = .5 * probs[0,1] + probs[0,2]
        prob_matrix[(j,i)] = .5 * probs[0,1] + probs[0,0]
  

In [36]:

#given a matrix of pairwise beliefs about who will win, lose between teams

#generate a distribution over outcomes of the tournament.

​

import numpy as np

from itertools import product, combinations, permutations

import heapq

import random

​

random.seed(1010101)

​

def rr_outcomes(n):

    '''Get all possible outcomes of a round robin tournament with n teams.

    Result will be an nxn matrix where outcome(i,j) = i beats j.

    Diagonals will be 0.'''

    num_games = (n*(n-1))/2

    outcomes = []

    for result in product([0,1], repeat=num_games):

        #result is 1-0 vector representing wins, losses for a set of games

        new_outcome = np.zeros((n,n))

        new_outcome[np.triu_indices(n, k=1)] = result

        new_outcome[np.tril_indices(n, k=-1)] = (1-new_outcome.T)[np.tril_indices(n, k=-1)]

        outcomes.append(new_outcome)

    return outcomes

​

def to_wl_ids(outcome, ids):

    '''Where outcome is an nxn 1-0 matrix for the outcome of a game,

    and ids is a list of n player ids, return two lists in the form winners,

    losers. Use this to index the probability array.'''

    loser_mat, winner_mat = np.meshgrid(group, group)

    return (winner_mat[outcome==1], loser_mat[outcome==1])

​

def _mapper(ids):

    qualified_countries = ['Egypt', 'Morocco', 'Nigeria', 'Senegal', 'Tunisia', 'Australia', 'Iran', 'Japan', 'Korea Republic', 'Saudi Arabia', 'Belgium', 'Croatia', 'Denmark', 'England', 'France', 'Germany', 'Iceland', 'Poland', 'Portugal', 'Russia', 'Serbia', 'Spain', 'Sweden', 'Switzerland', 'Costa Rica', 'Mexico', 'Panama', 'Argentina', 'Brazil', 'Colombia', 'Peru', 'Uruguay']

    return [qualified_countries[i] for i in ids]

#teams will be identified by row #s

#this should be a 32x32 matrix of win-loss probs

#winloss[a,b] = P(a beats b)

# wl = np.random.rand(32*32).reshape((32,32))

# wl = np.triu(wl,1)

# wl += np.tril(1-wl.T, -1)

​

#import prob matrix from machine learning stage

from get_probs import prob_matrix as wl

​

#group stage



[ 0.5  0.4  0.2  0.   0.6  0.5  0.6  0.5  0.5  0.5  0.   0.   0.   0.   0.
  0.   0.6  0.   0.   0.4  0.1  0.   0.2  0.   0.6  0.2  0.9  0.   0.   0.1
  0.6  0. ]
[ 0.6  0.5  0.4  0.1  0.9  0.7  0.9  0.6  0.6  0.7  0.   0.   0.   0.   0.
  0.   0.9  0.   0.   0.5  0.3  0.   0.3  0.   0.9  0.4  1.   0.   0.   0.2
  0.9  0. ]
[ 0.8  0.6  0.5  0.1  1.   0.9  0.9  0.8  0.7  0.9  0.   0.   0.1  0.   0.
  0.   1.   0.1  0.   0.6  0.4  0.   0.5  0.1  0.9  0.5  1.   0.   0.   0.3
  0.9  0. ]
[ 1.   0.9  0.9  0.5  1.   1.   1.   1.   1.   1.   0.   0.2  0.4  0.   0.
  0.   1.   0.3  0.   0.9  0.8  0.   0.9  0.4  1.   0.9  1.   0.   0.   0.8
  1.   0.2]
[ 0.4  0.1  0.   0.   0.5  0.5  0.5  0.3  0.3  0.5  0.   0.   0.   0.   0.
  0.   0.5  0.   0.   0.1  0.   0.   0.   0.   0.5  0.   0.5  0.   0.   0.
  0.5  0. ]
[ 0.5  0.3  0.1  0.   0.5  0.5  0.5  0.5  0.5  0.5  0.   0.   0.   0.   0.
  0.   0.5  0.   0.   0.3  0.1  0.   0.1  0.   0.5  0.1  0.6  0.   0.   0.
  0.5  0. ]
[ 0.4  0.1  0.1  0.   0.

In [42]:
from get_probs import prob_matrix as wl

In [47]:
matrix = wl.round(2)
pd.DataFrame(matrix).head(10)

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0.50,0.37,0.22,0.03,0.65,0.53,0.59,0.50,0.49,0.53,...,0.20,0.02,0.59,0.23,0.90,0.00,0.00,0.09,0.58,0.01
1,0.63,0.50,0.37,0.07,0.92,0.72,0.86,0.63,0.61,0.74,...,0.34,0.04,0.87,0.37,0.99,0.00,0.00,0.18,0.85,0.02
2,0.78,0.63,0.50,0.11,0.97,0.86,0.95,0.77,0.75,0.87,...,0.47,0.07,0.95,0.50,1.00,0.01,0.01,0.28,0.94,0.03
3,0.97,0.93,0.89,0.50,1.00,0.99,1.00,0.97,0.96,0.99,...,0.88,0.37,1.00,0.89,1.00,0.03,0.02,0.77,0.99,0.18
4,0.35,0.08,0.03,0.00,0.50,0.49,0.50,0.34,0.30,0.49,...,0.02,0.00,0.50,0.03,0.50,0.00,0.00,0.01,0.50,0.00
5,0.47,0.28,0.14,0.01,0.51,0.50,0.51,0.47,0.45,0.50,...,0.12,0.01,0.51,0.14,0.61,0.00,0.00,0.05,0.51,0.00
6,0.41,0.14,0.05,0.00,0.50,0.49,0.50,0.41,0.38,0.49,...,0.04,0.00,0.50,0.05,0.50,0.00,0.00,0.02,0.50,0.00
7,0.50,0.37,0.23,0.03,0.66,0.53,0.59,0.50,0.49,0.54,...,0.21,0.02,0.60,0.23,0.91,0.00,0.00,0.09,0.59,0.01
8,0.51,0.39,0.25,0.04,0.70,0.55,0.62,0.51,0.50,0.55,...,0.23,0.02,0.63,0.25,0.93,0.00,0.00,0.10,0.62,0.01
9,0.47,0.26,0.13,0.01,0.51,0.50,0.51,0.46,0.45,0.50,...,0.11,0.01,0.51,0.13,0.59,0.00,0.00,0.04,0.51,0.00


In [51]:
for g in groups: 
    for x in g:
        print x + ' & '
    print 

Russia & 
Saudi Arabia & 
Egypt & 
Uruguay & 

Portugal & 
Spain & 
Morocco & 
Iran & 

France & 
Australia & 
Peru & 
Denmark & 

Argentina & 
Iceland & 
Croatia & 
Nigeria & 

Brazil & 
Switzerland & 
Costa Rica & 
Serbia & 

Germany & 
Mexico & 
Sweden & 
Korea Republic & 

Belgium & 
Panama & 
Tunisia & 
England & 

Poland & 
Senegal & 
Colombia & 
Japan & 

